# Tagowanie części mowy - Pierwsze kroki: Praca z plikami tekstowymi, tworzenie słownika i obsługa nieznanych słów

W tym notebooku stworzysz słownik z oznaczonego zbioru danych i nauczysz się radzić sobie ze słowami, które nie są obecne w tym słowniku podczas pracy z innymi źródłami tekstu. Poza tym nauczysz się również jak:
 
- czytać pliki tekstowe
- pracować z defaultdict
- pracować z ciągami znaków (stringami)
 

In [1]:
import string
from collections import defaultdict

### Odczyt danych tekstowych

Oznaczony zbiór danych pobrany z Wall Street Journal znajduje się w pliku `WSJ_02-21.pos`. 

Aby odczytać ten plik, możesz użyć menedżera kontekstowego Pythona, używając słowa kluczowego `with` i określając nazwę pliku, który chcesz przeczytać. Aby faktycznie zapisać zawartość pliku do pamięci, należy użyć metody `readlines()` i zapisać jego wartość zwrotną w zmiennej. 

Menadżery kontekstowe Pythona są świetne, ponieważ nie trzeba wyraźnie zamykać połączenia z plikiem, robi się to pod spodem:

In [2]:
# Odczyt danych z pliku 'WSJ_02-21.pos' i zapis do zmiennej 'lines'
with open("WSJ_02-21.pos", 'r') as f:
    lines = f.readlines()

Aby sprawdzić zawartość zbioru danych wypiszmy pierwsze 5 wierszy:

In [3]:
# Print columns for reference
print("\t\tWord", "\tTag\n")

# Print first five lines of the dataset
for i in range(5):
    print(f'line number {i+1}: {lines[i]}')

		Word 	Tag

line number 1: In	IN

line number 2: an	DT

line number 3: Oct.	NNP

line number 4: 19	CD

line number 5: review	NN



Każdy wiersz w zbiorze danych ma słowo, po którym następuje odpowiedni tag. Ponieważ jednak wydruk został wykonany przy użyciu sformatowanego łańcucha, można wnioskować, że **word** i **tag** są oddzielone tabulatorem (lub kilkoma spacjami) i na końcu każdego wiersza znajduje się nowy wiersz (zauważ, że między każdym wierszem jest spacja). 

Jeśli chcesz zrozumieć znaczenie tych tagów, możesz zajrzeć [tutaj](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html).

Aby lepiej zrozumieć strukturę informacji w zbiorze danych, zaleca się wydrukowanie jej niesformatowanej wersji:

In [4]:
# Print first line (unformatted)
lines[0]

'In\tIN\n'

Rzeczywiście pomiędzy słowem i tagiem znajduje się tabulator a na końcu każdego wiersza jest nowa linia.

### Tworzenie słownika

Teraz, gdy rozumiesz, jak zbudowany jest zbiór danych, stworzysz z niego słownik. Słownik składa się z każdego słowa, które pojawiło się w zbiorze danych co najmniej 2 razy. 
W tym celu wykonaj poniższe kroki:
- Pobierz tylko słowa ze zbioru danych
- Użyj defaultdict, aby zliczyć ilość razy, kiedy pojawia się każde słowo.
- Przefiltruj dict tak, aby zawierało tylko te słowa, które pojawiły się co najmniej 2 razy
- Stwórzcie listę z przefiltrowanego dict
- Posortuj listę

W kroku 1 można wykorzystać fakt, że każde słowo i znacznik są oddzielone tabulatorem i że słowa zawsze znajdują się na pierwszym miejscu. Korzystając ze comprehension list, można w ten sposób stworzyć listę słów:

In [7]:
# z każdej linii ze zbioru danych wyciągamy tylko słowo
words = [line.split('\t')[0] for line in lines]

Krok 2 można wykonać z łatwością, używając `defaultdict`. W przypadku, gdy nie jesteś zaznajomiony z defaultdicts, są to specjalne rodzaje słowników, które **zwracają wartość "zero" danego typu, jeśli spróbujesz uzyskać dostęp do klucza, który nie istnieje**. Ponieważ zależy ci na częstotliwości występowania słów, powinieneś zdefiniować defaultdict z typem `int`. 

Teraz nie musisz się martwić o przypadek, gdy słowa nie ma w słowniku, ponieważ uzyskanie wartości dla tego klucza po prostu zwróci zero. Czy to nie jest fajne?

In [8]:
# Definiujemy defaultdict z typem 'int'
freq = defaultdict(int)

# zliczamy pojawienie się każdego słowa w zbiorze danych
for word in words:
    freq[word] += 1

Filtrowanie słownika `freq` może być wykonane przy użyciu list comprehension (czyż nie są one przydatne?). Należy odfiltrować słowa, które pojawiły się tylko raz, a także słowa, które są tylko znakiem nowej linii:

In [9]:
# tworzymy słownika ze słownika 'freq'
vocab = [k for k, v in freq.items() if (v > 1 and k != '\n')]

W końcu, metoda "sort" zajmie się ostatnim etapem. Zauważ, że zmienia ona listę bezpośrednio, więc nie ma potrzeby ponownego przypisywania zmiennej `vocab`:

In [10]:
# Sortujemy słownik
vocab.sort()

# Wypiszmy kilka przykładowych wartości ze słownika
for i in range(4000, 4005):
    print(vocab[i])

Early
Earnings
Earth
Earthquake
East


Teraz udało Ci się stworzyć słownik ze zbioru danych. **Wspaniała praca!** Słownik jest dość obszerny, więc nie będziemy go wypisywać w całości, ale możesz to zrobić, tworząc komórkę i uruchamiając coś w rodzaju `print(vocab)`. 

W tym momencie zazwyczaj będziesz zapisywał słownik do pliku do wykorzystania w przyszłości, ale to jest poza zakresem tego notebooka Jeśli jesteś ciekawy, to jest to bardzo podobne do tego, jak odczytuje ten plik na początku tego notebooka.


## Procesowanie nowych źródeł tekstu

### Radzenie sobie z nieznanymi słowami (spoza słownika)

Teraz, gdy masz już słownik, będziesz go używał podczas przetwarzania nowych źródeł tekstu. **Nowy tekst będzie zawierał słowa, które nie występują w obecnym słowniku**. Aby temu zaradzić, możesz po prostu sklasyfikować każde nowe słowo jako nieznane, ale możesz zrobić to lepiej, tworząc funkcję, która próbuje sklasyfikować typ każdego nieznanego słowa i przypisać mu odpowiedni `nieznany token`. 

Funkcja ta wykona następujące czynności sprawdzające i zwróci odpowiedni token:

   - Sprawdza, czy nieznane słowo zawiera dowolny znak będący cyfrą 
       - `--unk_digit--`
   - Sprawdź, czy nieznane słowo zawiera jakiś znak interpunkcyjny 
       - `--unk_punct--`
   - Sprawdź, czy nieznane słowo zawiera jakieś duże litery. 
       - `--unk_upper--`
   - Sprawdź, czy nieznany wyraz kończy się przyrostkiem, który może wskazywać na to, że jest odpowiednio rzeczownikiem, czasownikiem, przymiotnikiem lub przysłówkiem. 
        - return `--unk_noun--`, `--unk_verb--`, `--unk_adj--`, `--unk_adv--` 

Jeśli słowo nie spełni żadnego z powyższych warunków, to jego tokenem będzie zwykły `--unk--`. Warunki będą oceniane w tej samej kolejności, co tutaj wymienione. Więc jeśli słowo zawiera znak interpunkcyjny, ale nie zawiera cyfr, to będzie podlegało drugiemu warunkowi. Aby osiągnąć takie zachowanie, niektóre wyrażenia if/elif mogą być używane wraz z wczesnymi zwrotami. 

Funkcja ta jest zaimplementowana poniżej. Zauważ, że funkcja `any()` jest często używana. Zwraca `True` jeśli przynajmniej jeden z przypadków, który ocenia, jest `True`.

In [13]:
def assign_unk(word):
    """
    Przypisuje tokeny nieznanym słowom
    """
    
    # Znaki interpunkcyjne
    # Spróbuj wypisać je w oddzielnej komórce
    punct = set(string.punctuation)
    
    # przyrostki
    noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
    verb_suffix = ["ate", "ify", "ise", "ize"]
    adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
    adv_suffix = ["ward", "wards", "wise"]

    # Pętla po wszystkich znakach w słowie, sprawdzamy czy któryś ze znaków nie jest cyfrą
    if any(char.isdigit() for char in word):
        return "--unk_digit--"

    # Pętla po wszystkich znakach w słowie, sprawdzamy czy któryś ze znaków nie jest znakiem interpunkcyjnym
    elif any(char in punct for char in word):
        return "--unk_punct--"

    # Pętla po wszystkich znakach w słowie, sprawdzamy czy któryś ze znaków nie jest wielką literą
    elif any(char.isupper() for char in word):
        return "--unk_upper--"

    # Sprawdzamy czy słowo nie kończy się rzeczownikowym przyrostkiem
    elif any(word.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Sprawdzamy czy słowo nie kończy się czasownikowym przyrostkiem
    elif any(word.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Sprawdzamy czy słowo nie kończy się przymiotnikowym przyrostkiem
    elif any(word.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Sprawdzamy czy słowo nie kończy się przysłówkowym przyrostkiem
    elif any(word.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"
    
    # Jeżeli żaden z powyższych warunków nie jest spełniony to przypisujemy zwykły UNK
    return "--unk--"


In [12]:
punct = set(string.punctuation)
print(punct)

{'-', ';', '!', '{', '?', '"', '$', '~', '@', '&', '<', '*', ':', '#', '%', '^', '}', '`', ',', "'", '\\', '.', '(', '[', '>', '_', ']', ')', '/', '+', '|', '='}


Tworzony przez nas model oznaczający części mowy zawsze natrafi na słowa, których nie będzie w używanym przez nas słowniku. Poprzez powiększenie zbioru danych, włączając te `nieznane tokeny słów` pomagasz modelowu mieć lepsze wyobrażenie o odpowiednim tagu dla tych słów.

### Otrzymywanie poprawnego tagu dla słowa

Pozostaje tylko zaimplementować funkcję, która uzyska odpowiedni tag dla danego słowa, biorąc pod uwagę nieznane słowa. Ponieważ zbiór danych dostarcza każde słowo i tag w tym samym wierszu, a to, czy dane słowo jest znane, zależy od użytego słownika, te dwa elementy powinny być argumentami dla tej funkcji.

Funkcja ta powinna sprawdzić, czy wiersz jest pusty, a jeśli tak, to powinna zwrócić odpowiednio słowo i znacznik typu placeholder, `--n--` i `--s--`. 

Jeśli nie, to powinna przetworzyć wiersz, by zwrócić poprawną parę słów i znaczników, rozważając, czy słowo nie jest nieznane, w tym wypadku powinna być użyta funkcja `assign_unk()`.

Funkcja jest implementowana poniżej. Zauważ, że metoda `split()` może być użyta bez określania znaku rozdzielającego, w którym to przypadku będzie domyślna dla każdej spacji.

In [15]:
def get_word_tag(line, vocab):
    # jeżeli wiersz jest pusty to zwracamy placeholdery
    if not line.split():
        word = "--n--"
        tag = "--s--"
    else:
        # rozdzielamy wiersz, aby oddzielić słowo od tagu
        word, tag = line.split()
        # sprawdzamy czy słowo jest w słowniku
        if word not in vocab: 
            # przypisujemy tag dla nieznanego słowa
            word = assign_unk(word)
    return word, tag

Teraz możesz spróbować tej funkcji z kilkoma przykładami, aby sprawdzić, czy działa ona zgodnie z założeniami:

In [21]:
get_word_tag('\n', vocab)

('--n--', '--s--')

Ponieważ ten wiersz zawiera tylko znak nowej linii, funkcja zwraca słowo i tag zastępczy (placeholder)

In [22]:
get_word_tag('In\tIN\n', vocab)

('In', 'IN')

Ten wiersz jest poprawny, a funkcja wykonuje się prawidłowo i zwraca poprawną parę (słowo, tag).

In [23]:
get_word_tag('tardigrade\tNN\n', vocab)

('--unk--', 'NN')

Ten wiersz zawiera rzeczownik, który nie jest obecny w słowniku. 

Funkcja `assign_unk` nie wykrywa, że jest rzeczownikiem, więc zwraca `nieznany token`.

In [24]:
get_word_tag('scrutinize\tVB\n', vocab)

('--unk_verb--', 'VB')

Ten wiersz zawiera czasownik, który nie jest obecny w słownictwie. 

W tym przypadku `assign_unk` jest w stanie wykryć, że jest to czasownik, więc zwraca `nieznany token czasownika`.

**Gratuluję ukończenia tego notebooka!** Teraz powinieneś być bardziej zaznajomiony z pracą z danymi tekstowymi i lepiej rozumieć jak działa podstawowy model oznaczający części mowy.

**Kontynuuj dobrą robotę!**